In [1]:
import numpy as np 
import pandas as pd

In [2]:
df = pd.read_csv('MLS_Data.csv', sep=',')
df.head()

,Progress,Duration,Q_RecaptchaScore,Follow_Sports,Follow_Sports_Others,Special_Events,Freind_Family_Invitation,Famous_Player,Storyline_Rivalry,Halftime_Entertainment_Or_Shows,...,Other_online_livestream_plan_Others_Price,Age,Gender,Ethnicity,Ethnicity_Other,Employment_Status,Employment_Status_Other,Annual_Household_Income,Additional_Comments,Follow_Sports_Dummy
0,100,69,0.9,No,NaN,Somewhat Unlikely,Somewhat Likely,Somewhat Unlikely,Somewhat Unlikely,Somewhat Likely,...,NaN,25-34,Female,White or Caucasian,NaN,Employed part-time,NaN,"$25,000 - $49,999",NaN,0
1,100,58,0.9,No,NaN,Somewhat Likely,Somewhat Unlikely,Very Unlikely,Very Unlikely,Somewhat Likely,...,NaN,18-24,Male,White or Caucasian,NaN,Employed full-time,NaN,"$75,000 - $99,999",NaN,0
2,100,91,0.9,"Baseball,Ice Hockey,Soccer",NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,25-34,Male,White or Caucasian,NaN,Employed full-time,NaN,"$100,000 - $149,999",NaN,1
3,100,121,0.9,"Basketball,Ice Hockey",NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,35-44,Male,White or Caucasian,NaN,Employed full-time,NaN,"$150,000 or more",NaN,1
4,100,147,0.9,No,NaN,Somewhat Likely,Very Likely,Somewhat Unlikely,Somewhat Unlikely,Neutral,...,NaN,35-44,Female,White or Caucasian,NaN,Employed part-time,NaN,"$150,000 or more",NaN,0


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 169 entries, 0 to 168
Data columns (total 70 columns):
 #   Column                                                  Non-Null Count  Dtype  
---  ------                                                  --------------  -----  
 0   Progress                                                169 non-null    int64  
 1   Duration                                                169 non-null    int64  
 2   Q_RecaptchaScore                                        168 non-null    float64
 3   Follow_Sports                                           169 non-null    object 
 4   Follow_Sports_Others                                    44 non-null     object 
 5   Special_Events                                          37 non-null     object 
 6   Freind_Family_Invitation                                37 non-null     object 
 7   Famous_Player                                           37 non-null     object 
 8   Storyline_Rivalry                       

In [4]:
# group by followed sports
df_Follow_Sports = df['Follow_Sports'].str.lower().str.split(',').explode()
df_Follow_Sports = df_Follow_Sports.str.strip().replace('others(please specify', 'others')
df_Follow_Sports = df_Follow_Sports.str.strip().replace('and the answer can be more than one sport)', pd.NA).dropna()
df_Follow_Sports_counts = df_Follow_Sports.value_counts()

print(df_Follow_Sports_counts)
print(df_Follow_Sports_counts/137)

soccer        63
basketball    62
baseball      55
others        44
no            37
ice hockey    27
tennis        19
golf          10
Name: Follow_Sports, dtype: int64
soccer        0.459854
basketball    0.452555
baseball      0.401460
others        0.321168
no            0.270073
ice hockey    0.197080
tennis        0.138686
golf          0.072993
Name: Follow_Sports, dtype: float64


In [5]:
# other followed sports
df_Follow_Sports_Others = df.loc[:,"Follow_Sports_Others"].str.lower().str.split(', ')
df_Follow_Sports_Others = df_Follow_Sports_Others.dropna()

#football-related
df_Follow_Sports_Others = df_Follow_Sports_Others.apply(lambda x: [y if 'football' not in y else 'football' for y in x])
df_Follow_Sports_Others = df_Follow_Sports_Others.apply(lambda x: [y if 'nfl' not in y else 'football' for y in x])

#f1 = formula 1
df_Follow_Sports_Others = df_Follow_Sports_Others.apply(lambda x: [y.replace('f1', 'formula 1') for y in x])


df_Follow_Sports_Others_counts = df_Follow_Sports_Others.explode().value_counts()
print(df_Follow_Sports_Others_counts)

football          37
gymnastics         3
boxing             2
formula 1          2
ufc                2
curling            1
sailing            1
alpine skiing      1
volleyball         1
mma                1
skateboarding      1
equestrain         1
swimming           1
snowboarding       1
figure skating     1
Name: Follow_Sports_Others, dtype: int64


In [7]:
#intervals
#level
level = {
    'Very Likely': 5, 'Very Challenging': 5, 'Very Exciting': 5, 'Extremely Significant': 5,
    'Somewhat Likely': 4, 'Somewhat Challenging': 4, 'Somewhat Challenge': 4, 'Somewhat Exciting': 4, 'Somewhat Significant': 4,
    'Neutral': 3,
    'Somewhat Unlikely': 2, 'Slightly Challenging': 2, 'Slightly Exciting': 2, 'Slightly Significant': 2,
    'Slightly Significiant': 2,
    'Very Unlikely': 1, 'Not Challenging at all': 1, 'Not Exciting at all': 1, 'Not Significant at all': 1,
    'Not significant at all': 1
}
df.iloc[ : , 5:19 ] = df.iloc[:, 5:19].replace(level)
df.iloc[ : , 41:44 ] = df.iloc[:, 41:44].replace(level)


#age
Age = {'18-24': 1, '25-34': 2, '35-44': 3, '45-54': 4, '55-64': 5, '65 or over': 6 }
df.iloc[ : , 61 ] = df.iloc[:, 61].replace(Age)

#Annual_Household_Income
Annual_Household_Income = {
    'Under $25,000': 1, '$25,000 - $49,999': 2, '$50,000 - $74,999': 3,
    '$75,000 - $99,999': 4, '$100,000 - $149,999': 5, '$150,000 or more': 6,
    'Prefer not to say': None
}
df.iloc[ : , 67 ] = df.iloc[:, 67].replace(Annual_Household_Income)

mean_income = df['Annual_Household_Income'].mean()
df['Annual_Household_Income'].fillna(mean_income, inplace=True)

#MLS fan time 
MLS_fan_time = {
    'Less than a year': 1,
    '1-5 years': 2,
    '6-10 years': 3,
    'More than 10 years': 4,
    'Since its inception': 1
}
df.iloc[ : , 24:25 ] = df.iloc[:, 24:25].replace(MLS_fan_time)

In [9]:
#easy dummy
#gender
Gender = { "Male":1, "Female":0 }
df.iloc[ : , 62 ] = df.iloc[:, 62].replace(Gender)

#Follow_MLS_or_not
Follow_MLS = { "Yes":1, "No":0 }
df.iloc[ : , 21:22 ] = df.iloc[:, 21:22].replace(Follow_MLS)
df['Follow_MLS_or_not'].fillna(0, inplace=True)

In [36]:
#sport dummy
# Lowercase all the sports for consistency
#df['Follow_Sports'] = df['Follow_Sports'].str.lower().str.replace('no', '', regex=False)
#df['Follow_Sports_Others'] = df['Follow_Sports_Others'].str.lower().str.replace('no', '', regex=False)

# Combine both sports columns into one for easier processing
#df['All_Sports'] = df['Follow_Sports'].str.cat(df['Follow_Sports_Others'], sep=',', na_rep='')

# football-related
#def clean_and_unify_football_f1(sports_string):
#    sports_list = sports_string.split(',')
#    sports_list = [sport.strip() for sport in sports_list if sport.strip() != '']
#    sports_list = ['football' if 'football' in sport or 'nfl' in sport else sport for sport in sports_list]
#    sports_list = [sport.replace('f1', 'formula 1') for sport in sports_list]
#    return sports_list

#df['All_Sports'] = df['All_Sports'].apply(clean_and_unify_football_f1)

# Now we explode the lists into rows to use pd.get_dummies
#all_sports_exploded = df['All_Sports'].explode()
#sports_dummies = pd.get_dummies(all_sports_exploded)
# Group by the index and sum to avoid duplicating 1's for the same respondent
#sports_dummies = sports_dummies.groupby(sports_dummies.index).sum()
# only have 0's or 1's
#sports_dummies = sports_dummies.clip(upper=1)


#df_final = df.join(sports_dummies)
#df_final = df_final.drop(['others(please specify', 'and the answer can be more than one sport)'], axis=1)

,Progress,Duration,Q_RecaptchaScore,Follow_Sports,Follow_Sports_Others,Special_Events,Freind_Family_Invitation,Famous_Player,Storyline_Rivalry,Halftime_Entertainment_Or_Shows,...,ice hockey,mma,sailing,skateboarding,soccer,swboarding,swimming,tennis,ufc,volleyball
0,100,69,0.9,,NaN,2.0,4.0,2.0,2.0,4.0,...,0,0,0,0,0,0,0,0,0,0
1,100,58,0.9,,NaN,4.0,2.0,1.0,1.0,4.0,...,0,0,0,0,0,0,0,0,0,0
2,100,91,0.9,"baseball,ice hockey,soccer",NaN,NaN,NaN,NaN,NaN,NaN,...,1,0,0,0,1,0,0,0,0,0
3,100,121,0.9,"basketball,ice hockey",NaN,NaN,NaN,NaN,NaN,NaN,...,1,0,0,0,0,0,0,0,0,0
4,100,147,0.9,,NaN,4.0,5.0,2.0,2.0,3.0,...,0,0,0,0,0,0,0,0,0,0


In [15]:
# info about mls dummy
# Lowercase all the sports for consistency
df['Info_about_MLS'] = df['Info_about_MLS'].str.lower().str.split(',').replace('no', '', regex=False)
df['Info_about_MLS'] = df['Info_about_MLS'].str.strip().replace('others(please specify', 'others')
df['Info_about_MLS'] = df['Info_about_MLS'].str.strip().replace('and the answer can be more than one sport)', pd.NA).dropna()
df['All_Sports'] = df['Follow_Sports'].str.cat(df['Follow_Sports_Others'], sep=',', na_rep='')

# Now we explode the lists into rows to use pd.get_dummies
info = df['Info_about_MLS'].explode()
info_dummies = pd.get_dummies(info)
# Group by the index and sum to avoid duplicating 1's for the same respondent
info_dummies = info_dummies.groupby(info_dummies.index).sum()
# only have 0's or 1's
info_dummies = info_dummies.clip(upper=1)


df_final = df.join(info_dummies)


#Watch_MLS games
#How_Watch_Other_Pro_Sports
df['Watch_MLS games'] = df['Watch_MLS games'].str.lower().str.split(',').replace('no', '', regex=False)
df['How_Watch_Other_Pro_Sports'] = df['How_Watch_Other_Pro_Sports'].str.lower().str.split(',').replace('no', '', regex=False)

# Combine both sports columns into one for easier processing
df['All_Watching_Source'] = df['Watch_MLS games'].str.cat(df['How_Watch_Other_Pro_Sports'], sep=',', na_rep='')

all_source = df['All_Watching_Source'].explode()
source_dummies = pd.get_dummies(all_source)
# Group by the index and sum to avoid duplicating 1's for the same respondent
source_dummies = source_dummies.groupby(source_dummies.index).sum()
# only have 0's or 1's
source_dummies = source_dummies.clip(upper=1)


df_final = df.join(source_dummies)

AttributeError: Can only use .str accessor with string values!

In [14]:
df_final.to_csv('MLS_Data_Interval_2.csv', index=False)